In [86]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep

In [87]:
# getting CIK for every company in the s&p 500 from wikipedia

wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
cik_df = pd.read_html(wiki_url, header=0, index_col=0)[0]
cik_list = list(cik_df['CIK'])

In [100]:
# creating empty dataframe to append all other dataframes with 8-k links
doc_df = pd.DataFrame()

for cik in cik_list:
    
    try:
        
        # defining endpoint and parameters for every company in the s&p 
        url = 'https://www.sec.gov/cgi-bin/browse-edgar'
        params = {'action': 'getcompany', 
          'CIK': cik, 
          'type': '8-K', 
          'output':'xml', 
          'dateb': '20200430',
          'datea': '20150501',
          'start': '',
          'count': '100'}

        # getting response from EDGAR database
        sec_response = requests.get(url=url, params=params)

        # creating soup to parse xml
        soup = BeautifulSoup(sec_response.content, 'xml')

        # getting link to 8-k document
        urls = soup.findAll('filingHREF')
        html_list = []

        # html version of links
        for url in urls:
            url = url.string

            if url.split('.')[len(url.split('.')) - 1] == 'htm':
                txt_link = url + 'l'
                html_list.append(txt_link)
        
        html_list = pd.Series(html_list).astype(str)
        
        # list of links
        doc_list = html_list.str.replace('-index.html', '.txt').values.tolist()

        # creating dataframe to append the link of each company
        
        df = pd.DataFrame({'cik': [cik]*len(doc_list),
                   'txt_link': doc_list})

        doc_df = doc_df.append(df)

    except requests.exceptions.ConnectionError:
        sleep(.1)

In [109]:
doc_df.to_csv('D:/Python/lstm-stock-predictor/data/raw/8-k_links.csv', index=False)